# HW3.  Database Design

### Library Relations

- Member = {<span style="text-decoration:underline">memberID</span>, name, birthday}
- Borrow = {<span style="text-decoration:underline">borrowID</span><span style="text-decoration:underline">memberID</span>,<span style="text-decoration:underline">itemID</span>}
- BorrowTransactions = {<span style="text-decoration:underline">borrowID</span>, borrowDate, returnDate}
- Owes = {<span style="text-decoration:underline">fineID</span><span style="text-decoration:underline">memberID</span>,<span style="text-decoration:underline">itemID</span>}
- Fines = {<span style="text-decoration:underline">fineID</span>, status, amount}
- Item = {<span style="text-decoration:underline">itemID</span>, name, author, genre, status}
- History = {<span style="text-decoration:underline">itemID</span>, <span style="text-decoration:underline">recordID</span>}
- Records = {<span style="text-decoration:underline">recordID</span>, status, lastUpdated}
- Volunteers = {<span style="text-decoration:underline">memberID</span>,<span style="text-decoration:underline">staffID</span>}
- Works = {<span style="text-decoration:underline">memberID</span>,<span style="text-decoration:underline">staffID</span>}
- Staff = {<span style="text-decoration:underline">staffID</span>, name, position, employmentDate, wage}
- Hold = {{<span style="text-decoration:underline">staffID</span>,<span style="text-decoration:underline">eventID</span>}}
- Attends = {<span style="text-decoration:underline">memberID</span>, <span style="text-decoration:underline">eventID</span>}
- Events = {<span style="text-decoration:underline">eventID</span>, name, scheduledTime, scheduledDate, targetAudience}
- Located = {<span style="text-decoration:underline">eventID</span>, <span style="text-decoration:underline">roomNum</span>}
- Room = {<span style="text-decoration:underline">roomNum</span>, maxCap}

#### Creating Database

In [37]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [38]:
%sql sqlite:///library.db

In [39]:
%%sql

CREATE TABLE Member (
    memberID INTEGER PRIMARY KEY,
    name VARCHAR(100),
    birthday DATE,
    status VARCHAR(8) CHECK (status IN ('Active', 'Inactive'))
);

CREATE TABLE Borrow (
    borrowID INTEGER PRIMARY KEY,
    memberID INTEGER,
    itemID INTEGER,
    FOREIGN KEY (borrowID) REFERENCES BorrowTransactions(borrowID),
    FOREIGN KEY (memberID) REFERENCES Member(memberID),
    FOREIGN KEY (itemID) REFERENCES Item(itemID)
);


CREATE TABLE BorrowTransactions (
    borrowID INTEGER PRIMARY KEY,
    borrowDate DATE,
    returnDate DATE
);

CREATE TABLE Owes (
    fineID INTEGER PRIMARY KEY,
    memberID INTEGER,
    itemID INTEGER,
    FOREIGN KEY (fineID) REFERENCES Fines(fineID),
    FOREIGN KEY (memberID) REFERENCES Member(memberID),
    FOREIGN KEY (itemID) REFERENCES Item(itemID)
);

CREATE TABLE Fines (
    fineID INTEGER PRIMARY KEY,
    status VARCHAR(6) CHECK (status IN ('Paid', 'Unpaid')),
    amount DECIMAL(10,2) CHECK (amount >= 0)
);

CREATE TABLE Item (
    itemID INTEGER PRIMARY KEY,
    name VARCHAR(255),
    author VARCHAR(100),
    category VARCHAR(50),
    genre VARCHAR(50),
    status VARCHAR(11) CHECK (status IN ('Available', 'Unavailable'))
);

CREATE TABLE History (
    itemID INTEGER,
    recordID INTEGER,
    PRIMARY KEY (itemID, recordID),
    FOREIGN KEY (itemID) REFERENCES Item(itemID),
    FOREIGN KEY (recordID) REFERENCES Records(recordID)
);

CREATE TABLE Records (
    recordID INTEGER PRIMARY KEY,
    status VARCHAR(20) CHECK (status IN ('In System', 'Awaiting Approval', 'Archived')),
    lastUpdated DATE
);

CREATE TABLE Staff (
    memberID INTEGER,
    staffID INTEGER PRIMARY KEY,
    position VARCHAR(50),
    wage DECIMAL(10,2),
    employmentDate DATE,
    FOREIGN KEY (memberID) REFERENCES Member(memberID)
);

CREATE TABLE Volunteer (
    memberID INTEGER,
    volunteerID INTEGER PRIMARY KEY,
    employmentDate DATE,
    FOREIGN KEY (memberID) REFERENCES Member(memberID)
);

CREATE TABLE Hold (
    staffID INTEGER,
    eventID INTEGER,
    PRIMARY KEY (staffID, eventID),
    FOREIGN KEY (staffID) REFERENCES Staff(staffID),
    FOREIGN KEY (eventID) REFERENCES Events(eventID)
);

CREATE TABLE Events (
    eventID INTEGER PRIMARY KEY,
    name VARCHAR(100),
    scheduledTime TIME,
    scheduledDate DATE,
    targetAudience VARCHAR(100)
);

CREATE TABLE Located (
    eventID INTEGER,
    roomNum INTEGER,
    PRIMARY KEY (eventID, roomNum),
    FOREIGN KEY (eventID) REFERENCES Events(eventID),
    FOREIGN KEY (roomNum) REFERENCES Room(roomNum)
);

CREATE TABLE Room (
    roomNum INTEGER PRIMARY KEY,
    maxCap INTEGER
);

Running query in 'sqlite:///library.db'

++
||
++
++

#### Triggers

In [40]:
%%sql
CREATE TRIGGER update_fine_status
AFTER UPDATE ON Fines
FOR EACH ROW
BEGIN
    -- Automatically update fine status when the fine amount is paid
    UPDATE Fines
    SET status = 'Paid'
    WHERE fineID = NEW.fineID AND NEW.amount = 0;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [41]:
%%sql
CREATE TRIGGER generate_late_fine
AFTER INSERT ON BorrowTransactions
FOR EACH ROW
BEGIN
    -- Check if the return date is past the due date and generate a fine
    INSERT INTO Fines (memberID, itemID, status, amount)
    SELECT NEW.memberID, NEW.itemID, 'Unpaid', 10.00
    WHERE NEW.returnDate > (SELECT dueDate FROM Borrow WHERE borrowID = NEW.borrowID);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [42]:
%%sql
CREATE TRIGGER prevent_borrow_if_fine_unpaid
BEFORE INSERT ON Borrow
FOR EACH ROW
BEGIN
    -- Check if the member has an unpaid fine
    SELECT RAISE(ABORT, 'Cannot borrow item. Member has unpaid fines.')
    FROM Fines
    WHERE memberID = NEW.memberID AND status = 'Unpaid'
    LIMIT 1;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [43]:
%%sql
CREATE TRIGGER auto_archive_record
AFTER UPDATE ON Records
FOR EACH ROW
BEGIN
    -- Automatically archive record when status changes to 'Archived'
    SELECT RAISE(ABORT, 'Archiving record') 
    WHERE NEW.status = 'Archived' AND NOT EXISTS (SELECT 1 FROM ArchivedRecords WHERE recordID = NEW.recordID);

    -- Insert the record into the ArchivedRecords table
    INSERT INTO ArchivedRecords (recordID, lastUpdated)
    VALUES (NEW.recordID, CURRENT_DATE);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [44]:
%%sql
CREATE TRIGGER prevent_item_deletion_if_borrowed
BEFORE DELETE ON Item
FOR EACH ROW
BEGIN
    -- Prevent item deletion if it is still borrowed
    SELECT RAISE(ABORT, 'Item cannot be deleted as it is currently borrowed.')
    WHERE EXISTS (SELECT 1 FROM Borrow WHERE itemID = OLD.itemID);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [45]:
%%sql
CREATE TRIGGER set_member_id
BEFORE INSERT ON Member
FOR EACH ROW
WHEN NEW.memberID IS NULL
BEGIN
    UPDATE Member
    SET memberID = (
        CASE 
            WHEN (SELECT MAX(memberID) FROM Member) IS NULL THEN 100
            ELSE (SELECT MAX(memberID) FROM Member) + 1
        END
    )
    WHERE rowid = NEW.rowid;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [48]:
%%sql
PRAGMA table_info(Fines)

Running query in 'sqlite:///library.db'

cid,name,type,notnull,dflt_value,pk
0,fineID,INTEGER,0,None,1
1,status,VARCHAR(6),0,None,0
2,amount,"DECIMAL(10,2)",0,None,0


#### Inserts

In [47]:
%%sql
-- Insert data into Member
INSERT INTO Member (name, birthday, status) VALUES
('Alice Smith', '1990-01-15', 'Active'),
('Bob Johnson', '1985-03-22', 'Active'),
('Charlie Brown', '2000-07-09', 'Inactive'),
('Diana Ross', '1995-12-30', 'Active'),
('Ethan Hunt', '1992-08-18', 'Inactive'),
('Fiona Apple', '1998-06-25', 'Active'),
('George Lucas', '1980-11-05', 'Active'),
('Helen Keller', '1975-05-14', 'Inactive'),
('Isaac Newton', '1963-04-01', 'Active'),
('Jack Sparrow', '1989-09-19', 'Active');

-- Insert data into BorrowTransactions
INSERT INTO BorrowTransactions (borrowID, borrowDate, returnDate) VALUES
(1, '2024-01-01', '2024-01-15'),
(2, '2024-01-05', '2024-01-20'),
(3, '2024-01-10', '2024-01-25'),
(4, '2024-01-12', '2024-01-27'),
(5, '2024-01-15', '2024-01-30'),
(6, '2024-01-20', '2024-02-05'),
(7, '2024-01-25', '2024-02-10'),
(8, '2024-02-01', '2024-02-15'),
(9, '2024-02-05', '2024-02-20'),
(10, '2024-02-10', '2024-02-25');

-- Insert data into Borrow
INSERT INTO Borrow (borrowID, memberID, itemID) VALUES
(1, 100, 1), 
(2, 100, 2),
(3, 101, 3), 
(4, 102, 4), 
(5, 103, 5),
(6, 104, 6), 
(7, 105, 7), 
(8, 106, 8), 
(9, 107, 9), 
(10, 108, 10);

-- Insert data into Item
INSERT INTO Item (itemID, name, author, category, genre, status) VALUES
(1, 'Book A', 'Author A', 'Fiction', 'Fantasy', 'Available'),
(2, 'Book B', 'Author B', 'Non-Fiction', 'History', 'Unavailable'),
(3, 'Book C', 'Author C', 'Fiction', 'Sci-Fi', 'Available'),
(4, 'Book D', 'Author D', 'Non-Fiction', 'Biography', 'Available'),
(5, 'Book E', 'Author E', 'Fiction', 'Mystery', 'Unavailable'),
(6, 'Book F', 'Author F', 'Non-Fiction', 'Science', 'Available'),
(7, 'Book G', 'Author G', 'Fiction', 'Horror', 'Unavailable'),
(8, 'Book H', 'Author H', 'Non-Fiction', 'Philosophy', 'Available'),
(9, 'Book I', 'Author I', 'Fiction', 'Drama', 'Available'),
(10, 'Book J', 'Author J', 'Non-Fiction', 'Self-Help', 'Unavailable');



-- Insert data into Records
INSERT INTO Records (recordID, status, lastUpdated) VALUES
(1, 'In System', '2024-01-01'), (2, 'Awaiting Approval', '2024-01-05'),
(3, 'Archived', '2024-01-10'), (4, 'In System', '2024-01-15'),
(5, 'Awaiting Approval', '2024-01-20'), (6, 'Archived', '2024-01-25'),
(7, 'In System', '2024-02-01'), (8, 'Awaiting Approval', '2024-02-05'),
(9, 'Archived', '2024-02-10'), (10, 'In System', '2024-02-15');

-- Insert data into Events
INSERT INTO Events (eventID, name, scheduledTime, scheduledDate, targetAudience) VALUES
(1, 'Storytime', '10:00:00', '2024-03-01', 'Children'),
(2, 'Book Club', '14:00:00', '2024-03-02', 'Adults'),
(3, 'Author Talk', '16:00:00', '2024-03-03', 'Everyone'),
(4, 'Poetry Reading', '18:00:00', '2024-03-04', 'Teens'),
(5, 'History Lecture', '12:00:00', '2024-03-05', 'Adults'),
(6, 'Science Fair', '15:00:00', '2024-03-06', 'Students'),
(7, 'Art Exhibit', '11:00:00', '2024-03-07', 'Everyone'),
(8, 'Tech Workshop', '17:00:00', '2024-03-08', 'Teens'),
(9, 'Movie Night', '19:00:00', '2024-03-09', 'Everyone'),
(10, 'Music Concert', '20:00:00', '2024-03-10', 'Everyone');


Running query in 'sqlite:///library.db'

10 rows affected.

RuntimeError: (sqlite3.OperationalError) table Fines has no column named memberID
[SQL: INSERT INTO BorrowTransactions (borrowID, borrowDate, returnDate) VALUES
(1, '2024-01-01', '2024-01-15'),
(2, '2024-01-05', '2024-01-20'),
(3, '2024-01-10', '2024-01-25'),
(4, '2024-01-12', '2024-01-27'),
(5, '2024-01-15', '2024-01-30'),
(6, '2024-01-20', '2024-02-05'),
(7, '2024-01-25', '2024-02-10'),
(8, '2024-02-01', '2024-02-15'),
(9, '2024-02-05', '2024-02-20'),
(10, '2024-02-10', '2024-02-25');]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


#### Indexes

In [9]:
%%sql

CREATE INDEX ItemIndex ON Item(name, author);

Running query in 'sqlite:///library.db'

++
||
++
++

#### FOR TESTING PURPOSES
Delete tables if you need to

In [36]:
%%sql 
-- Disable foreign key checks temporarily
PRAGMA foreign_keys = OFF;

-- Drop all tables (starting with those with foreign keys)
DROP TABLE IF EXISTS Hold;
DROP TABLE IF EXISTS Located;
DROP TABLE IF EXISTS Events;
DROP TABLE IF EXISTS Staff;
DROP TABLE IF EXISTS Volunteer;
DROP TABLE IF EXISTS Borrow;
DROP TABLE IF EXISTS BorrowTransactions;
DROP TABLE IF EXISTS History;
DROP TABLE IF EXISTS Records;
DROP TABLE IF EXISTS Fines;
DROP TABLE IF EXISTS Owes;
DROP TABLE IF EXISTS Item;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Room;

-- Re-enable foreign key checks
PRAGMA foreign_keys = ON;


Running query in 'sqlite:///library.db'

++
||
++
++